In [1]:
import time
import csv
import copy
import pandas as pd
import argparse
import numpy as np
import random
from multiprocessing import *
import mplleaflet
import matplotlib.pyplot as plt
from collections import deque

from lib.OsrmEngine import *
from lib.Agents import *
from lib.Demand import *
from lib.Constants import *

In [2]:
exe_loc = './osrm-backend-5.6.0/build/osrm-routed'
map_loc = './osrm-backend-5.6.0/greater-london-latest.osrm'

osrm = OsrmEngine(exe_loc, map_loc)
osrm.start_server()
osrm = OsrmEngine(exe_loc, map_loc)
osrm.start_server()

The routing server "http://0.0.0.0:5000" is killed
The routing server "http://0.0.0.0:5000" starts running
The routing server "http://0.0.0.0:5000" is killed
The routing server "http://0.0.0.0:5000" starts running


In [3]:
model = Model(10, demand, V=20, K=2)
T = 0

In [7]:
stime = time.time()
for T in range(0,600,30):
    model.dispatch_at_time(osrm, T)
etime = time.time()
runtime = etime - stime

AMoD system: 20 vehicles of capacity 2; 360.0 trips/h
  at t = 0.000, 22 requests, in which 2 in queue
req 10 from (0.1252152, 51.3955701) to (0.1140902, 51.3871844) at t = 186.856
  latest pickup at t = 666.856, latest dropoff at t = 1198.056
  pickup at t = -1.000, dropoff at t = -1.000
req 17 from (0.0919035, 51.4175855) to (0.0712996, 51.3933688) at t = 225.147
  latest pickup at t = 705.147, latest dropoff at t = 1694.547
  pickup at t = -1.000, dropoff at t = -1.000
veh 0 at (0.0730442, 51.3625026) when t = 270.000; occupancy = 0/2;
  service distance/time travelled: 417.8, 60.0; rebalancing distance/time travelled: 0.0, 0.0
  has 2 leg(s), distance = 4927.6, duration = 516.8，cost = 604.5
    pickup: req 12 at (0.0885501, 51.3657524), distance = 1541.0, duration = 175.5
    dropoff: req 12 at (0.1070611, 51.3758154), distance = 3386.6, duration = 341.3
veh 1 at (0.0610961, 51.3662765) when t = 270.000; occupancy = 1/2;
  service distance/time travelled: 1718.6, 240.0; rebalancing

In [10]:
def print_results(model):
    count_reqs = 0
    count_served = 0
    wt = 0.0
    vt = 0.0
    for req in model.reqs:
        if req.Tr >= WARM_UP and req.Tr <= WARM_UP+SIMULATION:
            count_reqs += 1
            if not np.isclose(req.Td, -1.0):
                count_served += 1
                wt += (req.Tp - req.Tr)
                vt += (req.Td - req.Tp)
    wt /= count_served
    vt /= count_served
    
    vstt = 0.0
    vsdt = 0.0
    vrtt = 0.0
    vrdt = 0.0
    for veh in model.vehs:
        vstt += veh.Ts
        vsdt += veh.Ds
        vrtt += veh.Tr
        vrdt += veh.Dr
    vstt /= model.V
    vsdt /= model.V
    vrtt /= model.V
    vrdt /= model.V

    print("*"*80)
    print("Simulation Time: %d s" % SIMULATION)
    print("System Settings:")
    print("  - Fleet Size: %d; Capacity: %d" % (model.V, model.K))
    print("  - Demandn Rate: %.1f trips/h" % (3600.0/model.D))
    print("  - Dispatching Interval: %.1f s" % INTERVAL)
    print("Simulation Results:")
    print("  - Requests:")
    print("    + service rate: %.1f%% (%d/%d)" % (
        100.0*count_served/count_reqs, count_served, count_reqs))
    print("    + wait time: %.1f s" % (wt))
    print("    + in-vehicle time: %.1f s" % (vt))
    print("  - Vehicles:")
    print("    + vehicle service time travelled: %.1f s" % (vstt))
    print("    + vehicle service distance travelled: %.1f m" % (vsdt))
    print("    + vehicle service time percentage: %.1f%%" % (float(vsdt)/SIMULATION))
    print("    + vehicle rebalancing time travelled: %.1f s" % (vrtt))
    print("    + vehicle rebalancing distance travelled: %.1f m" % (vrdt))

print_results(model)

********************************************************************************
Simulation Time: 3600 s
System Settings:
  - Fleet Size: 20; Capacity: 2
  - Demandn Rate: 360.0 trips/h
  - Dispatching Interval: 30.0 s
Simulation Results:
  - Requests:
    + service rate: 8.6% (5/58)
    + wait time: 126.8 s
    + in-vehicle time: 200.7 s
  - Vehicles:
    + vehicle service time travelled: 407.2 s
    + vehicle service distance travelled: 3609.9 m
    + vehicle service time percentage: 1.0%
    + vehicle rebalancing time travelled: 0.0 s
    + vehicle rebalancing distance travelled: 0.0 m


In [ ]:
fig = plt.figure(figsize=(16,10))
model.draw()
mplleaflet.display(fig)

In [ ]:
osrm.kill_server()